# GPT2 as text classifier

Helpful source: https://drlee.io/fine-tuning-gpt-2-for-sentiment-analysis-94ebdd7b5b24

## Install Required Libraries


In [1]:
!pip install datasets # unified interface for accessing and working with various datasets (by hugging face)
!pip install -U accelerate # library to optimize and accelerate numerical computations
!pip install -U transformers # library by hugging face that gives easy access to pre-trained models, tokenizers, and tools for fine-tuning models

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 10.8 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 5.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-non

## Loading and Processing the Dataset

We load the dataset from Hugging Face. Each sample consists of one strings feature that stores the title as well as the (start of the) article-text. The label is the category that the article belongs to (world, sports, business, sci/tech). [Link](https://huggingface.co/datasets/ag_news/viewer/default/train) to explore the structure of the data.

In [2]:
from datasets import load_dataset

dataset = load_dataset('ag_news')

print(dataset)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})


Reduce the size of the dataset (to reduce training times) whilst ensuring that the original structure and distribution of the data is kept.

In [3]:
from datasets import Dataset, DatasetDict
import pandas as pd

def take_a_percentage_of_data(dataset, percentage=0.1, shuffle=True, random_state=None):
    # sort and group the dataset by label
    df = pd.DataFrame(dataset)
    df_sorted = df.sort_values(by='label')
    grouped_dfs = df_sorted.groupby('label')

    # ensure that proportions of the groups remains the same as in the original dataset
    filtered_dfs_per_group = []
    for label, group in grouped_dfs:
        num_samples_to_keep = int(len(group) * percentage)
        filtered_group = group.head(num_samples_to_keep)
        filtered_dfs_per_group.append(filtered_group)

    # concatenate (and shuffle) the filtered group-wise dataframes
    filtered_df = pd.concat(filtered_dfs_per_group)
    if shuffle:
        filtered_df = filtered_df.sample(frac=1, random_state=random_state)

    filtered_df.reset_index(drop=True, inplace=True) # resets the index of the DataFrame, drops the previous index column
    filtered_df_as_dict = filtered_df.to_dict(orient='list')
    filtered_dataset = Dataset.from_dict(filtered_df_as_dict)
    return filtered_dataset

dataset_train_1percent = take_a_percentage_of_data(dataset['train'], percentage=0.01)
dataset_test_1percent = take_a_percentage_of_data(dataset['test'], percentage=0.01)

dataset_1percent = DatasetDict({
    'train': dataset_train_1percent,
    'test': dataset_test_1percent
}) # combine the shortened datasets back into the old structure.

print(dataset_1percent)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1200
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 76
    })
})


## Tokenizing the dataset

Tokenize the dataset in the exact same way as the GPT-2 model.

In [4]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token # padding tokens added to sequences will be represented by an end-of-sequence token
def tokenize_function(examples):
    #return tokenizer(examples["text"], padding="max_length", truncation=True)
    return tokenizer(examples["text"], padding="max_length")

tokenized_dataset = dataset_1percent.map(tokenize_function, batched=True) # performed in batches to increase performance

print(tokenized_dataset) # tokenization adds two features: 'input_ids' (the tokenized representation of 'text') as well as 'attention_mask', which ensures that the model does not attend to padding tokens added during tokenization

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

Map:   0%|          | 0/76 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1200
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 76
    })
})


## Loading the Pre-trained GPT2-Model

Load the pre-trained GPT2-Model for sequence classification.

In [5]:
from transformers import GPT2ForSequenceClassification

gpt2_model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=4) # our gpt2-model should distinguish between 4 labels, adds a final fully connected layers with 4 output neurons.

print(gpt2_model)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=4, bias=False)
)


## Implementing the Gaussian Adaptive Attention Block

Download the package provided by the paper.

In [6]:
!pip3 install gaussian-adaptive-attention

In [7]:
from gaussian_adaptive_attention import MultiHeadGaussianAdaptiveAttention
import torch
import copy

gaussian_model = copy.deepcopy(gpt2_model)

class MultiHeadGaussianAdaptiveAttentionWrapper(torch.nn.Module):
    def __init__(self, config, num_gaussians=10, norm_axis=1):
        super().__init__()
        self.attention = MultiHeadGaussianAdaptiveAttention(
            norm_axis=norm_axis, # same as example of the researchers (dario had = 2)
            num_heads=config.n_head,
            num_gaussians=num_gaussians,
            padding_value=config.eos_token_id,
            eps=config.layer_norm_epsilon
        )

    def forward(self, hidden_states, **kwargs):
        # Pass arguments using **kwargs to the underlying attention mechanism
        attention_output = self.attention(hidden_states)
        return (hidden_states,) + tuple(attention_output)  # Ensure the return value is a tuple


# Replace the attention mechanism in each transformer block
for block in gaussian_model.transformer.h: # accessing each transformer blocks within the GPT-2 model
    block.attn = MultiHeadGaussianAdaptiveAttentionWrapper(config=gaussian_model.config, num_gaussians=5, norm_axis=1) # and replacing the attention module with the Gaussian attention block.

# print(gaussian_model.config)


Define a GPT-2 transformer block that uses a Gaussian attention mechanism rather than an attention function that is based on matrix multiplications

In [8]:
print(gaussian_model)
print(gpt2_model)

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): MultiHeadGaussianAdaptiveAttentionWrapper(
          (attention): MultiHeadGaussianAdaptiveAttention(
            (attention_heads): ModuleList(
              (0-11): 12 x GaussianAdaptiveAttention()
            )
          )
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=4, bias=False)
)
GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (

## Freezing Parameters

Experiment

In [16]:
for param in gaussian_model.transformer.parameters():
    param.requires_grad = False

for param in gaussian_model.score.parameters():
    param.requires_grad = True

for block in gaussian_model.transformer.h:
  for param in block.attn.parameters():
      param.requires_grad = True


In [10]:
trainable_params = sum(p.numel() for p in gaussian_model.parameters() if p.requires_grad)
frozen_params = sum(p.numel() for p in gaussian_model.parameters() if not p.requires_grad)

print(f"Trainable Parameters: {trainable_params}")
print(f"Frozen Parameters: {frozen_params}")

Trainable Parameters: 4512
Frozen Parameters: 96091392


## Test

1. The Normal GPT2-Model
2. The GPT2-Model with GAAM

In [15]:
input_ids = torch.randint(0, gpt2_model.config.vocab_size, (1, 512))
labels = torch.tensor([1]).unsqueeze(0)

outputs = gpt2_model(input_ids=input_ids, labels=labels)
loss, logits = outputs['loss'], outputs['logits']
print(f"GPT2 with regular attention mechanism: Loss = {round(loss.item(), 4)}, logits = {logits.detach()}")

outputs = gaussian_model(input_ids=input_ids, labels=labels)
loss, logits = outputs['loss'], outputs['logits']
print(f"GPT2 with Gaussian attention mechanism: Loss = {round(loss.item(), 4)}, logits = {logits.detach()}")


GPT2 with regular attention mechanism: Loss = 7.7526, logits = tensor([[-5.3510, -0.9909,  6.7612, -4.6367]])
GPT2 with Gaussian attention mechanism: Loss = 3.789, logits = tensor([[-2.2199, -0.7506,  2.9999, -1.5780]])


## Training

Training the GPT2-Model with Gaussian attention

In [19]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=1e-3,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=5,
    # weight_decay=0.01,
    use_cpu = False,
    no_cuda = False
)

trainer = Trainer(
    model=gaussian_model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
)


In [20]:
model_name = "gpt2-ag_news-1percent-frozen"

trainer.train()
trainer.save_model(model_name)
# Save the tokenizer used by the model as well
tokenizer.save_pretrained(model_name)


Step,Training Loss
500,1.468000
1000,1.411700
1500,1.410500
2000,1.393900
2500,1.401700
3000,1.405500
3500,1.401800
4000,1.394100
4500,1.404400
5000,1.398600


('gpt2-ag_news-1percent-frozen/tokenizer_config.json',
 'gpt2-ag_news-1percent-frozen/special_tokens_map.json',
 'gpt2-ag_news-1percent-frozen/vocab.json',
 'gpt2-ag_news-1percent-frozen/merges.txt',
 'gpt2-ag_news-1percent-frozen/added_tokens.json')

In [21]:
trainer.evaluate()

{'eval_loss': 1.3863848447799683,
 'eval_runtime': 13.2415,
 'eval_samples_per_second': 5.74,
 'eval_steps_per_second': 5.74,
 'epoch': 5.0}

## Calculating the accuracy

In [25]:
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

gaussian_model.to(device)

def predict_label(sentence):
    inputs = tokenizer(sentence, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = gaussian_model(**inputs)
        prediction = outputs.logits.argmax(-1).item()

    return prediction

def calculate_accuracy(model, data):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for item in data:
            prediction = predict_label(item['text'])
            label = item['label']
            total += 1
            if label == prediction:
              correct += 1

    return correct / total

data = dataset_1percent['test']

# Calculate accuracy
accuracy = calculate_accuracy(gaussian_model, data)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.2500


## Second Training Cycle with Unfrozen Layers

Loading the previously trained model and unfreezing the layers.

In [27]:

unfrozen_gaussian_model = copy.deepcopy(gaussian_model)
print(unfrozen_gaussian_model)

# unfreeze layers
for param in unfrozen_gaussian_model.transformer.parameters():
    param.requires_grad = True

# check
trainable_params = sum(p.numel() for p in unfrozen_gaussian_model.parameters() if p.requires_grad)
print(f"Trainable Parameters: {trainable_params}")

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): MultiHeadGaussianAdaptiveAttentionWrapper(
          (attention): MultiHeadGaussianAdaptiveAttention(
            (attention_heads): ModuleList(
              (0-11): 12 x GaussianAdaptiveAttention()
            )
          )
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=4, bias=False)
)
Trainable Parameters: 96095904


Defining the new parameters for the second training cycle (smaller learning rate, including weight decay)

In [30]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=5,
    weight_decay=0.01,
    use_cpu = False,
    no_cuda = False
)

trainer = Trainer(
    model=unfrozen_gaussian_model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
)


Starting the second training cycle.

In [29]:
trainer.train()

Step,Training Loss
500,1.405000
1000,1.399500
1500,1.395700
2000,1.391200
2500,1.396400
3000,1.389600
3500,1.390000
4000,1.390000
4500,1.393400
5000,1.387900


TrainOutput(global_step=6000, training_loss=1.3929286905924478, metrics={'train_runtime': 2274.1474, 'train_samples_per_second': 2.638, 'train_steps_per_second': 2.638, 'total_flos': 2090634706944000.0, 'train_loss': 1.3929286905924478, 'epoch': 5.0})

In [31]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

unfrozen_gaussian_model.to(device)

# Test the function with an example from the dataset
text = dataset['test'][1]['text']
label = dataset['test'][1]['label']
print("Predicted label:", predict_label(text))
print("Expected label:", label)


Predicted label: 2
Expected label: 3


In [35]:
from torch.utils.data import DataLoader

def calculate_accuracy(model, data):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for item in data:
            prediction = predict_label(item['text'])
            label = item['label']
            total += 1
            if label == prediction:
              correct += 1

    return correct / total

data = dataset_1percent['test']

# Calculate accuracy
accuracy = calculate_accuracy(unfrozen_gaussian_model, data)
print(f"Accuracy: {accuracy:.4f}")

# problem: exact same accuracy score as before, so something must be wrong

Accuracy: 0.2500
